<p style="font-family: Arial; font-size:2.75em;color:purple; font-style:bold"><br>

Avrupa Futbol Takımlarının Regresyon (Regression)  <br><br> Modeli İle Analizi -Scikit-learn

<br><br></p>


Dataset' i <a href="https://www.kaggle.com">Kaggle</a>'dan indirebilirsiniz. 

+25,000 maç
+10,000 futbolcu
11 Ülkenin şampiyon kulüpleri
2008 - 2016 Sezonu
Takım ve Futbolcuların Özellikleri
Ayrıntılı Maç Detayları (Gol Tipleri, pozisyonlar, korner, çalım , faul, kartlar vs...)  +10,000 maç için.

<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

Gerekli Küphanelerin İçe Aktarımı<br><br></p>


In [1]:
import sqlite3
import pandas as pd 
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

Veritabanından Veriyi Pandas DataFrame'ine Çekme
<br><br></p>


In [2]:
# sqlite3 ile veritabanı bağlantısı kurma
cnx = sqlite3.connect('database.sqlite')
df = pd.read_sql_query("SELECT * FROM Player_Attributes", cnx)

In [3]:
df.head()

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,218353,505942,2016-02-18 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,2,218353,505942,2015-11-19 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,3,218353,505942,2015-09-21 00:00:00,62.0,66.0,right,medium,medium,49.0,...,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,4,218353,505942,2015-03-20 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,5,218353,505942,2007-02-22 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0


In [4]:
silmeden_once = df.shape[0]
silmeden_once

183978

In [5]:
df.columns

Index(['id', 'player_fifa_api_id', 'player_api_id', 'date', 'overall_rating',
       'potential', 'preferred_foot', 'attacking_work_rate',
       'defensive_work_rate', 'crossing', 'finishing', 'heading_accuracy',
       'short_passing', 'volleys', 'dribbling', 'curve', 'free_kick_accuracy',
       'long_passing', 'ball_control', 'acceleration', 'sprint_speed',
       'agility', 'reactions', 'balance', 'shot_power', 'jumping', 'stamina',
       'strength', 'long_shots', 'aggression', 'interceptions', 'positioning',
       'vision', 'penalties', 'marking', 'standing_tackle', 'sliding_tackle',
       'gk_diving', 'gk_handling', 'gk_kicking', 'gk_positioning',
       'gk_reflexes'],
      dtype='object')

<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

Analizde Kullanacağımız Özellikleri (Features) Tanımlama
<br><br></p>


In [6]:
features = [
       'potential', 'crossing', 'finishing', 'heading_accuracy',
       'short_passing', 'volleys', 'dribbling', 'curve', 'free_kick_accuracy',
       'long_passing', 'ball_control', 'acceleration', 'sprint_speed',
       'agility', 'reactions', 'balance', 'shot_power', 'jumping', 'stamina',
       'strength', 'long_shots', 'aggression', 'interceptions', 'positioning',
       'vision', 'penalties', 'marking', 'standing_tackle', 'sliding_tackle',
       'gk_diving', 'gk_handling', 'gk_kicking', 'gk_positioning',
       'gk_reflexes']

<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

Y (Target-Sonuç) Tanımlama
<br><br></p>


In [7]:
target = ['overall_rating']

<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>
Data Temizleme<br><br></p>


In [8]:
bos_satir_sayisi = df.isnull().any(axis=1).sum()
bos_satir_sayisi

3624

In [9]:
df = df.dropna()
sildikten_sonra = df.shape[0]
sildikten_sonra

180354

In [10]:
#Bu işlemin sonucu bos_satir_sayisi = 3624 değerine eşit olmalı
silmeden_once - sildikten_sonra 

3624

Gördüğünüz gibi sonuç istediğimiz değere eşit çıktı

<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

Özellikleri (Features) ve Target Değerlerini Ayrıştıma
<br><br></p>


In [11]:
X = df[features]

In [12]:
y = df[target]

In [13]:
X.shape

(180354, 34)

In [14]:
X.iloc[2]

potential             66.0
crossing              49.0
finishing             44.0
heading_accuracy      71.0
short_passing         61.0
volleys               44.0
dribbling             51.0
curve                 45.0
free_kick_accuracy    39.0
long_passing          64.0
ball_control          49.0
acceleration          60.0
sprint_speed          64.0
agility               59.0
reactions             47.0
balance               65.0
shot_power            55.0
jumping               58.0
stamina               54.0
strength              76.0
long_shots            35.0
aggression            63.0
interceptions         41.0
positioning           45.0
vision                54.0
penalties             48.0
marking               65.0
standing_tackle       66.0
sliding_tackle        69.0
gk_diving              6.0
gk_handling           11.0
gk_kicking            10.0
gk_positioning         8.0
gk_reflexes            8.0
Name: 2, dtype: float64

Sonuç (Target) değerlerine bakalım.

In [15]:
y.head(50)

,overall_rating
0,67.0
1,67.0
2,62.0
3,61.0
4,61.0
5,74.0
6,74.0
7,73.0
8,73.0
9,73.0


<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

Dataset' i Eğitim (Train) ve Test Kümelerine Ayrıma <br><br></p>


In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=324)

<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

(1) Lineer Regresyon (Linear Regression): 
<br><br></p>


In [17]:
regression = LinearRegression()
regression.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

Lineer Regresyon Modeli İle Tahmin Yapma
<br><br></p>


In [18]:
y_prediction = regression.predict(X_test)
y_prediction

array([[ 66.51284879],
       [ 79.77234615],
       [ 66.57371825],
       ..., 
       [ 69.23780133],
       [ 64.58351696],
       [ 73.6881185 ]])

<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

Tahmin Edilmesi Gereken Sonuç (Target) Değerinin Ortalamısı Nedir?  
<br><br></p>


In [19]:
y_test.describe()

,overall_rating
count,59517.000000
mean,68.635818
std,7.041297
min,33.000000
25%,64.000000
50%,69.000000
75%,73.000000
max,94.000000


68.63 oranında bir ortalamaya sahip. Tahmin değerlerimizinde bu civarda çıkmasını bekliyoruz.

<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

Lineer Regresyon Modelinin Doğruluğunu Root Mean Square Error Kullanarak Bulma

<br><br></p>


Eğer sonuç 0 çıkarsa Modelimiz mükemmel bir tahmin yapmıştır.

In [20]:
rmse = sqrt(mean_squared_error(y_true = y_test, y_pred = y_prediction))

In [21]:
print(rmse)

2.805303046855223


<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

(2) Karar Ağacı (Decision Tree):
<br><br></p>


In [22]:
regression = DecisionTreeRegressor(max_depth=20)
regression.fit(X_train, y_train)

DecisionTreeRegressor(criterion='mse', max_depth=20, max_features=None,
           max_leaf_nodes=None, min_impurity_split=1e-07,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, presort=False, random_state=None,
           splitter='best')

<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

Karar Ağacı Modeli İle Tahmin Yapma
<br><br></p>


In [23]:
y_prediction = regression.predict(X_test)
y_prediction

array([ 62.        ,  84.        ,  62.38666667, ...,  71.        ,
        62.        ,  73.        ])

<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

For comparision: What is the mean of the expected target value in test set ?
<br><br></p>


In [24]:
y_test.describe()

,overall_rating
count,59517.000000
mean,68.635818
std,7.041297
min,33.000000
25%,64.000000
50%,69.000000
75%,73.000000
max,94.000000


<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

Karar Ağacı Modelinin Doğruluğunu Root Mean Square Error Kullanarak Bulma

<br><br></p>


In [25]:
rmse = sqrt(mean_squared_error(y_true = y_test, y_pred = y_prediction))

In [26]:
print(rmse)

1.461295383115322


## Sonuç:

#### Karar Ağacı Modeli (Decision Tree) , Linear REgression Modeline göre daha iyi bir sonuç verdi.